In [10]:
import os
import numpy as np 
import pandas as pd 
from subprocess import check_output
import matplotlib.pyplot as plt

# Data Load

In [27]:
sub_path = "data/submissions"
all_files = os.listdir(sub_path)
all_files

['stacks',
 'submission_efficient_net.csv',
 'submission_fastai_352px_full.csv',
 'submission_fastai_original.csv',
 'submission_inceptionv3.csv']

In [28]:
selected_files = ['submission_fastai_352px_full.csv',
                  'stacks/leaderboard_subs_mean.csv']

In [29]:
# Read and concatenate submissions
outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in selected_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "rnsa" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
ncol = concat_sub.shape[1]
concat_sub.head()

/home/christian/anaconda3/envs/fastai_dev/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,index,rnsa0,rnsa1
0,ID_000012eaf_any,0.002541,0.069530
1,ID_000012eaf_epidural,0.000157,0.001824
2,ID_000012eaf_intraparenchymal,0.000422,0.013128
3,ID_000012eaf_intraventricular,0.000100,0.003252
4,ID_000012eaf_subarachnoid,0.000400,0.007577


In [30]:
# check correlation
concat_sub.iloc[:,1:ncol].corr()

,rnsa0,rnsa1
rnsa0,1.000000,0.963934
rnsa1,0.963934,1.000000


In [31]:
concat_sub.rename(index=str, columns={"index": "ID"}, inplace=True)

In [32]:
# get the data fields ready for stacking
concat_sub['rsna_max'] = concat_sub.iloc[:, 1:ncol].max(axis=1)
concat_sub['rsna_min'] = concat_sub.iloc[:, 1:ncol].min(axis=1)
concat_sub['rsna_mean'] = concat_sub.iloc[:, 1:ncol].mean(axis=1)
concat_sub['rsna_median'] = concat_sub.iloc[:, 1:ncol].median(axis=1)

In [33]:
concat_sub.iloc[:, 1:ncol].describe()

,rnsa0,rnsa1
count,471270.000000,471270.000000
mean,0.050788,0.054087
std,0.185367,0.178716
min,0.000100,0.000033
25%,0.000100,0.000322
50%,0.000191,0.001603
75%,0.002463,0.010520
max,0.999000,0.999577


# Median Stacking

In [18]:
concat_sub['Label'] = concat_sub['rsna_median']
concat_sub[['ID', 'Label']].to_csv('data/submissions/stacks/subs_median.csv', index=False)

# Mean Stacking

In [34]:
concat_sub['Label'] = concat_sub['rsna_mean']
concat_sub[['ID', 'Label']].to_csv('data/submissions/stacks/subs_mean_fullandlb.csv', index=False)

### Weighted Average

In [58]:
selected_files

['submission_efficient_net.csv',
 'submission_inceptionv3.csv',
 'submission_original.csv']

In [59]:
sub_efficient_net = pd.read_csv(os.path.join(sub_path, selected_files[0]), index_col=0)
sub_inceptionv3 = pd.read_csv(os.path.join(sub_path, selected_files[1]), index_col=0)
sub_fastai = pd.read_csv(os.path.join(sub_path, selected_files[2]), index_col=0)

In [60]:
cc = sub_fastai.copy()

In [61]:
cc['Label'] = (0.2*sub_fastai['Label']
             + 0.4*sub_inceptionv3['Label']
             + 0.4*sub_efficient_net['Label'])

In [62]:
print(cc['Label'].mean()-sub_fastai['Label'].mean())
print(cc['Label'].mean()-sub_inceptionv3['Label'].mean())
print(cc['Label'].mean()-sub_efficient_net['Label'].mean())

0.004855821950315385
-0.000620620574337892
-0.0018072904008283944


In [63]:
cc.to_csv('data/submissions/stacks/leaderboard_weighted_average2.csv')

In [56]:
cc.head()

,Label
ID,
ID_e3674b189_any,0.002339
ID_e3674b189_epidural,0.000028
ID_e3674b189_intraparenchymal,0.000302
ID_e3674b189_intraventricular,0.000107
ID_e3674b189_subarachnoid,0.001340
